## 简答题
1. 如果训练集有100万个实例，训练决策树（无约束）大致的深度是多少？
2. 通常来说，子节点的基尼杂质是高于还是低于其父节点？是通常更高/更低？还是永远更高/更低？
3. 如果决策树过拟合训练集，减少max_depth是否为一个好主意？
4. 如果决策树对训练集欠拟合，尝试缩放输入特征是否为一个好主意？
5. 如果在给定的训练集上训练决策树需要一个小时，那么如果将特征数量变为两倍，训练大约需要多少时间？
6. 如果在包含100万个实例的训练集上训练决策树需要一个小时，那么在包含1000万个实例的训练集上训练决策树，大概需要多长时间？提示：考虑CART算法的计算复杂度。

## 编程题

1. 为 新月形 数据集训练并微调一棵决策树。

a. 使用make_moons(n_samples=10000，noise=0.4)生成一个 新月形 数据集。

b. 使用train_test_split()拆分训练集和测试集。

c. 使用交叉验证的网格搜索（在GridSearchCV类的帮助下）为Decision-TreeClassifier找到适合的超参数值。提示：尝试max_leaf_nodes的多种值。

d. 使用超参数对整个训练集进行训练，并测量模型在测试集上的性能。你应该得到约85%～87%的准确率。

In [1]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.datasets import make_moons
import numpy as np

X,y = make_moons(n_samples=10000,noise=0.4,random_state=42)
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.2,random_state=42)

# param_grid = {
#     'max_depth': [3, 5, 7, 10, None],
#     'max_leaf_nodes':   2**np.arange(1, 10),
#     'min_samples_leaf': range(1, 50, 2),
#     'min_samples_split': [2, 5, 10],
#     'criterion': ['gini', 'entropy'],
#     'max_features': [None, 'sqrt', 'log2']
# }
param_grid = {
    'criterion': ['gini', 'entropy'],
    'max_leaf_nodes':   2**np.arange(1, 10),
    'min_samples_leaf': range(1, 50, 2)
}
gscv = GridSearchCV(DecisionTreeClassifier(random_state=42), param_grid, cv=5, scoring='accuracy', n_jobs=-1, verbose=1)
gscv.fit(X_train, y_train)
print("最佳参数:", gscv.best_params_)
print("最佳交叉验证准确率:", gscv.best_score_)
test_accuracy = gscv.score(X_test, y_test)
print("测试集准确率:", test_accuracy)

Fitting 5 folds for each of 450 candidates, totalling 2250 fits
最佳参数: {'criterion': 'entropy', 'max_leaf_nodes': 128, 'min_samples_leaf': 45}
最佳交叉验证准确率: 0.8612499999999998
测试集准确率: 0.867


2. 按照以下步骤种植森林。

a.继续之前的练习，生产1000个训练集子集，每个子集包含随机挑选的100个实例。提示：使用Scikit-Learn的ShuffleSplit类来实现。

b.使用前面得到的最佳超参数值，在每个子集上训练一棵决策树。在测试集上评估这1000棵决策树。因为训练集更小，所以这些决策树的表现可能比第一棵决策树要差一些，只能达到约80%的精度。

c.见证奇迹的时刻到了。用每个测试集实例，生成1000棵决策树的预测，然后仅保留次数最频繁的预测［可以使用SciPy的mode()函数］。这样你在测试集上可获得大多数投票的预测结果。

d.评估测试集上的这些预测，你得到的准确率应该比第一个模型更高（高出0.5%～1.5%）。恭喜，你已经训练出了一个随机森林分类器！

In [10]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import ShuffleSplit

shuffler = ShuffleSplit(n_splits=100,train_size=0.8,test_size=None,random_state=42)
forests = []
scores = []
predictions = []
best_parmas = gscv.best_params_

for train_index, test_index in shuffler.split(X_train):
    tree_clf = DecisionTreeClassifier(**best_parmas,max_features='sqrt',
                                      random_state=np.random.randint(10000))

    tree_clf.fit(X_train[train_index], y_train[train_index])
    forests.append(tree_clf)

    tree_pred = tree_clf.predict(X_test)
    predictions.append(tree_pred)
    scores.append(accuracy_score(y_test, tree_pred))

np.mean(scores)

0.8629350000000001

In [11]:
from scipy.stats import mode
forest_pred, _ = mode(predictions, axis=0)
accuracy_score(y_test, forest_pred)

0.873

训练 1000 棵有差异的决策树...
单棵树平均准确率: 0.772
集成模型准确率: 0.788
